
### M2_SII_23_24:
<center><h1>TP4 RCR2:</h1></center>
<center><h2>Logique possibiliste qualitative</h2></center>
---
FERKOU SARAH - KHEMISSI MAROUA



In [2]:
!pip install pycosat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycosat: filename=pycosat-0.6.6-cp310-cp310-linux_x86_64.whl size=169341 sha256=531b2ae483e7007547f60fca71419594c40db7437a1278314baf4b25f167429a
  Stored in directory: /root/.cache/pip/wheels/63/29/df/b8c22ca5812e2d7b342269a53add280b5bad42a540f34c3dc1
Successfully built pycosat


In [3]:
import random
import os
import pycosat

In [5]:
class BaseConnaissance:
    number_of_clause = 0
    number_of_variable = 0
    evidence_length = 0
    var_list = []
    max_clause_length = 0

    def __init__(self, nb_clause, nb_variable, length, max_number):
        self.number_of_clause = nb_clause
        self.number_of_variable = nb_variable
        self.evidence_length = length
        self.max_clause_length = max_number
        for i in range(self.number_of_variable):
            self.var_list.append(i + 1)
            self.var_list.append(-(i + 1))

    def get_evidence(self):
        return random.sample(self.var_list, self.evidence_length)

    def get_sigma(self):
        with open("temp.txt", "w") as sigma_file:
            for i in range(self.number_of_clause):
                ran = random.randrange(1, self.max_clause_length + 1, 1)
                temp_list = random.sample(self.var_list, ran)
                weight = random.random()
                weight = float(int(weight * 100)) / 100
                chaine = "" + str(weight) + " "
                for j in range(len(temp_list)):
                    chaine = chaine + str(temp_list[j]) + " "
                sigma_file.write(chaine + "\n")
                temp_list = []


In [6]:
class Sigma:
    lower = 0
    upper = 0
    weights = []
    formulas = []
    strates_weights = []

    def __init__(self, path_file):
        with open(path_file) as file:
            for line in file:
                information = line.split()
                self.weights.append(float(information[0]))
                self.formulas.append([int(information[i]) for i in range(1, len(information))])
        self.length = len(self.weights)

    def get_length(self):
        return self.length

    def get_weights(self):
        return self.weights

    def get_formulas(self):
        return self.formulas

    def sort_weights(self):
        sorted_data = sorted(zip(self.weights, self.formulas), reverse=True)
        self.weights, self.formulas = zip(*sorted_data)

    def compute_strates(self):
        self.upper = len(set(self.weights))
        self.strates_weights = sorted(set(self.weights))

    def get_strates_number(self):
        return self.upper

    def get_strates_weights(self):
        return self.strates_weights

    def get_preprocessed_formulas(self, sub_formulas):
        dict_cor = {}
        returned_formulas = []
        cpt = 1

        for form in sub_formulas:
            temp_form = []
            for pred in form:
                if pred not in dict_cor:
                    dict_cor[pred] = cpt
                    dict_cor[-pred] = -cpt
                    cpt += 1
                temp_form.append(dict_cor[pred])
            returned_formulas.append(temp_form)

        return returned_formulas


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
pip install prettytable

In [10]:
from prettytable import PrettyTable
choice = 1
if choice == 1:
    base_de_connaissance = Sigma("/content/drive/MyDrive/data/BD.wcnf")
    evidence = [3, 4, -7]
else:
    nb_clause = 8
    nb_variable = 5
    length = 3
    max_number = 4
    gen = BaseConnaissance(nb_clause, nb_variable, length, max_number)
    gen.get_sigma()
    base_de_connaissance = Sigma("temp.txt")
    evidence = gen.get_evidence()

base_de_connaissance.sort_weights()
base_de_connaissance.compute_strates()
iteration = 1

while base_de_connaissance.lower < base_de_connaissance.upper:
    r = int((base_de_connaissance.lower + base_de_connaissance.upper + 1) / 2)

    liste = base_de_connaissance.get_weights()
    value_of_r = -1
    for i in range(len(liste)):
        if base_de_connaissance.get_strates_weights()[r - 1] > liste[i]:
            value_of_r = i
            break
    if value_of_r == -1:
        value_of_r = len(liste) - 1

    for j in range(len(liste)):
        if base_de_connaissance.get_strates_weights()[base_de_connaissance.upper - 1] == liste[j]:
            valueU = j
            break

    cnf = base_de_connaissance.formulas[valueU:value_of_r]

    for i in range(len(evidence)):
        liste = []
        liste.append(-1 * evidence[i])
        cnf = list(base_de_connaissance.get_preprocessed_formulas(cnf))


    cnf = base_de_connaissance.get_preprocessed_formulas(cnf)

    result = pycosat.solve(cnf)
    if type(result) == type([]):
        base_de_connaissance.upper = r - 1
    else:
        base_de_connaissance.lower = r

    iteration = iteration + 1

Val = base_de_connaissance.get_strates_weights()[r - 1]

with open("/content/drive/MyDrive/data/BD.wcnf") as f:
    content = f.read().splitlines()


# Création d'une table PrettyTable
table = PrettyTable()
table.field_names = ["Poids", "PropositionP1", "PropositionP2"]  # Remplacez les noms des colonnes par ceux appropriés

# Ajout des lignes à la table
for line in content:
    row_values = line.split()
    # Remplir les colonnes manquantes avec des chaînes vides
    while len(row_values) < len(table.field_names):
        row_values.append('')
    table.add_row(row_values)

# Affichage de la table
print(table)


print("**L'évidence de : ", evidence)
print("**Le resultat : Val(" + str(evidence) + ", Sigma) = " + str(Val))


+-------+---------------+---------------+
| Poids | PropositionP1 | PropositionP2 |
+-------+---------------+---------------+
|  0.6  |       3       |       2       |
|  0.57 |       4       |       3       |
|  0.9  |       -7      |               |
|  0.1  |       4       |               |
|  0.0  |       8       |               |
|  0.4  |       9       |               |
+-------+---------------+---------------+
**L'évidence de :  [3, 4, -7]
**Le resultat : Val([3, 4, -7], Sigma) = 0.0
